# 👶 The Little Baby

> A barebones GPT-style LLM implementation — pure Python, zero dependencies.

In [ ]:
import os
cores = os.cpu_count() or 8
print(f"Detected CPU cores: {cores}")
if "OMP_NUM_THREADS" not in os.environ:
    os.environ["OMP_NUM_THREADS"] = str(cores)
if "MKL_NUM_THREADS" not in os.environ:
    os.environ["MKL_NUM_THREADS"] = str(cores)
if "OPENBLAS_NUM_THREADS" not in os.environ:
    os.environ["OPENBLAS_NUM_THREADS"] = str(cores)
if "NUMEXPR_NUM_THREADS" not in os.environ:
    os.environ["NUMEXPR_NUM_THREADS"] = str(cores)

In [ ]:
import math
import uuid
import json
import time
import numpy as np
from datetime import datetime as dt

In [ ]:
########################
# Runtime helpers
########################

def is_valid_guid(guid_str):
    try:
        val = uuid.UUID(guid_str)
        return str(val) == guid_str  # Ensures exact format match
    except ValueError:
        return False

def from_file(file_path, file_mode):
    try:
        if file_mode == "plain":
            with open(file_path, "r") as f:
                file = f.read()
                return file
        elif file_mode == "json":
            with open(file_path, "r") as f:
                file = json.load(f)
                return file
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        exit(1)
        

def to_file(file_path, file_mode, content):
    try:
        if file_mode == "plain":
            with open(file_path, "w", encoding="utf-8") as f:
                f.write(content)
        elif file_mode == "json":
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(content, f, indent=4)
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        exit(1)

In [ ]:
########################
# Tokenizer Class
########################

class CharTokenizer:
    """
    Character-level tokenizer for text processing.
    Separates encoding/decoding logic for better modularity.
    """
    def __init__(self, text=None, vocab=None, stoi=None, itos=None):
        self.vocab = None
        self.vocab_size = None
        self.itos = None
        self.stoi = None

    def init(self, text):
        self.vocab = sorted(list(set(text)))
        self.vocab_size = len(self.vocab)
        self.itos = {i: c for i, c in enumerate(self.vocab)}
        self.stoi = {c: i for i, c in enumerate(self.vocab)}

    def expand_vocabilary(self, text):
        """ Expand the tokenizer vocabulary with new characters from the text. """
        new_chars = set(text) - set(self.vocab)
        if new_chars:
            print(f"Found {len(new_chars)} new characters: {new_chars}")
            # Add new characters to existing vocabulary and sort everything
            all_chars = set(self.vocab) | new_chars  # Union of sets
            self.vocab = sorted(list(all_chars))
            self.vocab_size = len(self.vocab)
            self.itos = {i: c for i, c in enumerate(self.vocab)}
            self.stoi = {c: i for i, c in enumerate(self.vocab)}

    def fit(self, text):
        """ Fit the tokenizer to the provided text. """
        if self.vocab is None:
            self.init(text)
        else:
            self.expand_vocabilary(text)

    def encode(self, text):
        """ Convert string to list of token IDs. """        
        return [self.stoi[c] for c in text]
    
    def decode(self, token_ids):
        """ Convert list of token IDs to string. """
        return ''.join([self.itos[i] for i in token_ids])
    
    def to_dict(self):
        """ Export tokenizer state as dictionary for JSON serialization. """
        return {
            'vocab': self.vocab,
            'vocab_size': self.vocab_size
        }
    
    def from_dict(self, tokenizer_dict):
        """ Load tokenizer from dictionary. """
        self.vocab=tokenizer_dict['vocab']
        self.vocab_size=len(self.vocab)
        self.itos={int(k): v for k, v in enumerate(self.vocab)} # Convert keys back to integers
        self.stoi={str(v): k for k, v in enumerate(self.vocab)} # Convert keys back to strings

    def tokenize(self, input_path):
        """ Tokenize input text and return data. """
        text = from_file(input_path, "plain")
        
        # Fit the tokenizer to the text
        self.fit(text)

        # Encode the text
        data = self.encode(text)
        split = int(0.9 * len(data))
        train_data = data[:split]
        val_data = data[split:]

        return train_data, val_data
    
    def prepare_data(self, data, n_ctx):
        """ Prepares data for training by creating input-output pairs. """

        X, y = [], []
        # Ensure there's enough data for at least one full context length + 1 for target
        if len(data) < n_ctx + 1:
            # Return empty arrays with correct shapes for concatenation later
            return np.array([], dtype=np.int32).reshape(0, n_ctx), np.array([], dtype=np.int32).reshape(0, n_ctx)

        for i in range(0, len(data) - n_ctx):
            X.append(data[i:i+n_ctx])
            y.append(data[i+1:i+n_ctx+1])
        return np.array(X, dtype=np.int32), np.array(y, dtype=np.int32)
    
    def get_batches(self, X, y, b_size, shuffle=True):
        """ Generates batches of data for training. """
        
        N = X.shape[0]
        if N == 0: # Handle empty data case
            return # Yield nothing if no data
        indices = np.arange(N)
        if shuffle:
            np.random.shuffle(indices)
        for i in range(0, N, b_size):
            batch_idx = indices[i:i+b_size]
            yield X[batch_idx], y[batch_idx]

In [ ]:
########################
# Base Module 
########################

class Module:
    """
    Base class for all neural network modules.
    Manages parameters and training/evaluation mode.
    """
    def __init__(self):
        self.setting = True # Default mode for modules
        self._parameters = [] # List to store trainable parameters

    def parameters(self):
        """
        Returns a list of all trainable parameters in the module.
        This method should be overridden by subclasses.
        """
        return self._parameters

    def set(self, mode=True):
        """
        Sets the module and all its sub-modules to training mode.
        If mode is False, sets to evaluation mode.
        """
        self.setting = mode
        # Recursively set training mode for any sub-modules
        for param in self._parameters: # Iterate through parameters, which might be other Modules
            if isinstance(param, Module):
                param.set(mode)
            # For actual numpy arrays (weights/biases), the `training` flag is used by Dropout
            # and other layers that behave differently in train/eval.

    def eval(self):
        """Sets the module to evaluation mode."""
        self.set(False)

In [ ]:
########################
# Helper Functions
########################

# GELU activation and its derivative
def gelu(x):
    """Gaussian Error Linear Unit (GELU) activation function."""
    return 0.5 * x * (1.0 + np.tanh(math.sqrt(2/math.pi) * (x + 0.044715 * np.power(x, 3))))

def gelu_prime(x):
    """Derivative of the GELU activation function."""
    # This is an approximation of the derivative, common in practice.
    # The exact derivative involves erf, which is complex.
    # This simplified version uses the derivative of tanh.
    k = math.sqrt(2/math.pi) * (x + 0.044715 * np.power(x, 3))
    sech_sq = 1 / np.cosh(k)**2 # sech^2(x) = 1 / cosh^2(x)
    k_prime = math.sqrt(2/math.pi) * (1 + 3 * 0.044715 * np.power(x, 2))
    return 0.5 * (1 + np.tanh(k)) + 0.5 * x * sech_sq * k_prime

# Softmax (along given axis)
def softmax(x, axis=-1):
    """Computes softmax probabilities along a given axis for numerical stability."""
    x_max = np.max(x, axis=axis, keepdims=True)
    e_x = np.exp(x - x_max)
    return e_x / np.sum(e_x, axis=axis, keepdims=True)

In [ ]:
########################
# Layers
########################

class Embedding(Module):
    """
    A simple Embedding layer.
    Maps integer indices to dense vectors.
    """
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        # Initialize weights with small random values
        self.weight = np.random.randn(num_embeddings, embedding_dim) * 0.02
        self._parameters = [self.weight] # Register weight as a parameter

    def forward(self, x):
        """
        Forward pass for Embedding layer.
        x: input indices (e.g., token IDs), shape (B, T) or (T,)
        Returns: embedded vectors, shape (B, T, embedding_dim) or (T, embedding_dim)
        """
        self._cache = x # Store input indices for backward pass
        return self.weight[x]

    def backward(self, grad_output):
        """
        Backward pass for Embedding layer.
        grad_output: gradient from the subsequent layer, shape (B, T, embedding_dim)
        Returns: (grad_input, [grad_weight])
        """
        x = self._cache # Retrieve input indices
        grad_weight = np.zeros_like(self.weight) # Initialize gradient for weights

        # Accumulate gradients for each embedding used.
        # This is a sparse update: only update rows corresponding to input indices.
        # if x.ndim == 1: # Handle (T,) input case
        #     for i, idx in enumerate(x):
        #         grad_weight[idx] += grad_output[i]
        # else: # Handle (B, T) input case
        #     for b in range(x.shape[0]):
        #         for t in range(x.shape[1]):
        #             grad_weight[x[b, t]] += grad_output[b, t]

        # Efficiently accumulate gradients using np.add.at
        # Remove loops in Embedding.backward (lets BLAS + vectorized add handle parallelism)
        grad_weight = np.zeros_like(self.weight)
        if x.ndim == 1:
            np.add.at(grad_weight, x, grad_output)
        else:
            np.add.at(grad_weight, x.reshape(-1), grad_output.reshape(-1, grad_output.shape[-1]))

        # For embedding layer, there's no gradient to pass back to the input (it's just indices).
        return None, [grad_weight]

class Linear(Module):
    """
    A simple Linear (fully connected) layer.
    Performs y = x @ W + b.
    """
    def __init__(self, in_features, out_features, bias=True):
        super().__init__()
        # Initialize weights with small random values
        self.weight = np.random.randn(in_features, out_features) * 0.02
        self.bias = np.zeros(out_features) if bias else None
        self._parameters = [self.weight]
        if self.bias is not None:
            self._parameters.append(self.bias)

    def forward(self, x):
        """
        Forward pass for Linear layer.
        x: input tensor, shape (..., in_features)
        Returns: output tensor, shape (..., out_features)
        """
        self._cache = x # Store input for backward pass
        out = x.dot(self.weight)
        if self.bias is not None:
            out = out + self.bias
        return out

    def backward(self, grad_output):
        """
        Backward pass for Linear layer.
        grad_output: gradient from the subsequent layer, shape (..., out_features)
        Returns: (grad_input, [grad_weight, grad_bias])
        """
        x = self._cache # Retrieve input
        original_x_shape = x.shape

        # Reshape x and grad_output to 2D for matrix multiplication for gradients
        # This handles arbitrary leading dimensions (B, T, ...)
        x_reshaped = x.reshape(-1, original_x_shape[-1]) # (N, in_features)
        grad_output_reshaped = grad_output.reshape(-1, grad_output.shape[-1]) # (N, out_features)

        # Gradient for weight: dL/dW = x.T @ dL/dy
        grad_weight = x_reshaped.T @ grad_output_reshaped

        grad_bias = None
        if self.bias is not None:
            # Gradient for bias: dL/db = sum(dL/dy) along all dimensions except the last
            grad_bias = np.sum(grad_output_reshaped, axis=0)

        # Gradient for input: dL/dx = dL/dy @ W.T
        grad_input = grad_output_reshaped @ self.weight.T
        grad_input = grad_input.reshape(original_x_shape) # Reshape back to original input shape

        param_grads = [grad_weight]
        if grad_bias is not None:
            param_grads.append(grad_bias)

        return grad_input, param_grads

class LayerNorm(Module):
    """
    Layer Normalization layer.
    Normalizes features across the last dimension.
    """
    def __init__(self, dims, eps=1e-5):
        super().__init__()
        self.eps = eps
        self.gamma = np.ones(dims)  # Learnable scaling parameter
        self.beta = np.zeros(dims)  # Learnable shifting parameter
        self._parameters = [self.gamma, self.beta]

    def forward(self, x):
        """
        Forward pass for LayerNorm.
        x: input tensor, shape (..., dims)
        Returns: normalized tensor, same shape as x
        """
        mean = x.mean(axis=-1, keepdims=True)
        var = x.var(axis=-1, keepdims=True)
        std = np.sqrt(var + self.eps)
        x_norm = (x - mean) / std # Normalized input
        out = self.gamma * x_norm + self.beta

        # Store intermediate values for backward pass
        self._cache = (x, mean, std, x_norm)
        return out

    def backward(self, grad_output):
        """
        Backward pass for LayerNorm.
        grad_output: gradient from subsequent layer, same shape as forward output.
        Returns: (grad_input, [grad_gamma, grad_beta])
        """
        x, mean, std, x_norm = self._cache

        # Gradients for gamma and beta (sum over all dimensions except the last)
        grad_gamma = np.sum(grad_output * x_norm, axis=tuple(range(grad_output.ndim - 1)))
        grad_beta = np.sum(grad_output, axis=tuple(range(grad_output.ndim - 1)))

        # Gradient for x_norm (before gamma/beta scaling)
        grad_x_norm = grad_output * self.gamma

        # Gradient for x (through normalization formula)
        # This is a common and numerically stable way to compute LayerNorm's grad_x
        N = x.shape[-1] # Number of features in the last dimension

        # Part 1: Gradient through (x - mean) / std
        grad_x = grad_x_norm / std

        # Part 2: Gradient through std (which comes from var)
        grad_var = np.sum(grad_x_norm * (x - mean) * (-0.5) * (std**(-3)), axis=-1, keepdims=True)

        # Part 3: Gradient through mean
        grad_mean = np.sum(grad_x_norm * (-1 / std), axis=-1, keepdims=True) + grad_var * (-2 / N) * np.sum(x - mean, axis=-1, keepdims=True)

        grad_x += (2 / N) * grad_var * (x - mean)
        grad_x += (1 / N) * grad_mean

        return grad_x, [grad_gamma, grad_beta]

class Dropout(Module):
    """
    Dropout layer.
    Randomly sets a fraction of input units to zero during training.
    """
    def __init__(self, p):
        super().__init__()
        self.p = p # Dropout probability

    def forward(self, x):
        """
        Forward pass for Dropout.
        x: input tensor
        Returns: output tensor with dropout applied (if training)
        """
        if self.setting and self.p > 0:
            # Create a mask: True for elements to keep, False for elements to drop
            # Scale by 1/(1-p) during training (inverted dropout)
            mask = (np.random.rand(*x.shape) >= self.p) / (1.0 - self.p)
            self._cache = mask # Store mask for backward pass
            return x * mask
        self._cache = None # No mask if not training or p=0
        return x

    def backward(self, grad_output):
        """
        Backward pass for Dropout.
        grad_output: gradient from subsequent layer.
        Returns: (grad_input, []) - no parameters to update.
        """
        if self._cache is not None: # Only apply mask if dropout was active in forward
            mask = self._cache
            grad_input = grad_output * mask
        else: # If dropout was not active, simply pass gradient through
            grad_input = grad_output
        return grad_input, [] # Dropout has no parameters


In [ ]:
########################
# Attention Mechanism
########################

class MultiHeadAttention(Module):
    """
    Multi-Head Self-Attention mechanism.
    Computes attention scores and combines information from different "heads".
    """
    def __init__(self, n_emb, head_size, n_heads, n_ctx, dropout_p):
        super().__init__()
        self.n_emb = n_emb
        self.head_size = head_size  # total projection dim (e.g. 128)
        self.n_heads = n_heads
        self.d_k = head_size // n_heads  # dimension per head (e.g. 32)

        # Linear projections for Query, Key, Value
        self.q_proj = Linear(n_emb, head_size, bias=False)
        self.k_proj = Linear(n_emb, head_size, bias=False)
        self.v_proj = Linear(n_emb, head_size, bias=False)

        # Output projection
        self.c_proj = Linear(head_size, n_emb)

        self.attn_dropout = Dropout(dropout_p)
        self.resid_dropout = Dropout(dropout_p)

        # Causal mask to prevent looking ahead in sequence (for decoder-only models)
        causal_mask = np.triu(np.ones((n_ctx, n_ctx)) * -1e9, k=1)
        self.causal_mask = causal_mask

        # KV cache for inference
        self.kv_cache = None

    def clear_cache(self):
        """Clear the KV cache."""
        self.kv_cache = None

    def parameters(self):
        """Returns all parameters of the attention module."""
        return (self.q_proj.parameters() +
                self.k_proj.parameters() +
                self.v_proj.parameters() +
                self.c_proj.parameters())

    def set(self, mode=True):
        """Sets the attention module and its sub-modules to training/eval mode."""
        super().set(mode) # Call base Module set to set self.set
        self.q_proj.set(mode)
        self.k_proj.set(mode)
        self.v_proj.set(mode)
        self.c_proj.set(mode)
        self.attn_dropout.set(mode)
        self.resid_dropout.set(mode)

        # Clear cache when switching to training mode
        if mode:
            self.clear_cache()

    def forward(self, x, use_cache):
        """
        Forward pass for MultiHeadAttention.
        x: input tensor, shape (B, T, n_emb)
        Returns: output tensor, shape (B, T, n_emb)
        """
        B, T, _ = x.shape

        # Project input to Q, K, V
        Q_orig = self.q_proj.forward(x)  # (B, T, head_size)
        K_orig = self.k_proj.forward(x)
        V_orig = self.v_proj.forward(x)

        # Helper function to split heads and transpose
        def split_heads(z):
            B_s, T_s, H_s = z.shape
            z = z.reshape(B_s, T_s, self.n_heads, self.d_k)
            return z.transpose(0, 2, 1, 3) # (B, n_heads, T, d_k)

        Q = split_heads(Q_orig)
        K_new = split_heads(K_orig)
        V_new = split_heads(V_orig)

        if use_cache and not self.setting:  # Only use cache during inference
            if self.kv_cache is not None:
                # Concatenate with cached K, V
                K_cached, V_cached = self.kv_cache
                K = np.concatenate([K_cached, K_new], axis=2)  # Concat along sequence dimension
                V = np.concatenate([V_cached, V_new], axis=2)
            else:
                K = K_new
                V = V_new
            
            # Update cache with new K, V
            self.kv_cache = (K, V)
        else:
            # Training mode or cache disabled
            K = K_new
            V = V_new

        # Get actual sequence length for attention computation
        actual_seq_len = K.shape[2]
        
        # Compute scaled dot-product attention scores
        # (B, n_heads, T, d_k) @ (B, n_heads, d_k, actual_seq_len) -> (B, n_heads, T, actual_seq_len)
        scores = np.matmul(Q, K.transpose(0, 1, 3, 2)) / math.sqrt(self.d_k)

        # Apply causal mask (prevents attending to future tokens)
        # Adjust mask for the actual sequence lengths
        if use_cache and T == 1 and actual_seq_len > 1:
            # For single token generation, create a mask for the last position
            # attending to all previous positions (including itself)
            mask = np.zeros((1, actual_seq_len))
            # No masking needed for single token attending to past
        else:
            # Normal case: create causal mask for current sequence length
            mask = self.causal_mask[:T, :actual_seq_len]
        
        masked_scores = scores + mask

        attn_weights = softmax(masked_scores, axis=-1)
        attn_weights_dropped = self.attn_dropout.forward(attn_weights)

        # Compute weighted sum of values
        # (B, n_heads, T, T) @ (B, n_heads, T, d_k) -> (B, n_heads, T, d_k)
        o = np.matmul(attn_weights_dropped, V)

        # Recombine heads: transpose and reshape back to (B, T, head_size)
        o_combined = o.transpose(0, 2, 1, 3).reshape(B, T, self.head_size)

        # Final linear projection
        out = self.c_proj.forward(o_combined)
        out_dropped = self.resid_dropout.forward(out)

        # Store all intermediate values needed for backward pass (unchanged for training)
        if self.setting:  # Only cache for backward pass during training
            self._cache = (x, Q_orig, K_orig, V_orig, Q, K_new, V_new, scores, masked_scores, attn_weights, attn_weights_dropped, o, o_combined)
            
        return out_dropped

    def backward(self, grad_output):
        """
        Backward pass for MultiHeadAttention.
        grad_output: gradient from the subsequent layer, shape (B, T, n_emb)
        Returns: (grad_input, list_of_param_grads)
        """
        (x, Q_orig, K_orig, V_orig, Q, K, V, scores, masked_scores, attn_weights, attn_weights_dropped, o, o_combined) = self._cache

        # Gradients will be collected in the order of self.parameters(): q_proj, k_proj, v_proj, c_proj
        current_mha_param_grads = []

        # 1. Backward through resid_dropout
        grad_out_dropped, _ = self.resid_dropout.backward(grad_output) # Dropout has no params

        # 2. Backward through c_proj (output linear layer)
        grad_o_combined, c_proj_grads = self.c_proj.backward(grad_out_dropped)

        # 3. Undo reshape/transpose for o_combined to get grad_o
        # grad_o_combined: (B, T, head_size)
        # grad_o: (B, n_heads, T, d_k)
        B, T, H = grad_o_combined.shape
        grad_o = grad_o_combined.reshape(B, T, self.n_heads, self.d_k).transpose(0, 2, 1, 3)

        # 4. Backward through matmul(attn_weights_dropped, V)
        # o = A @ V  => dL/dA = dL/do @ V.T, dL/dV = A.T @ dL/do
        grad_attn_weights_dropped = np.matmul(grad_o, V.transpose(0, 1, 3, 2))
        grad_V = np.matmul(attn_weights_dropped.transpose(0, 1, 3, 2), grad_o)

        # 5. Backward through attn_dropout
        grad_attn_weights, _ = self.attn_dropout.backward(grad_attn_weights_dropped)

        # 6. Backward through softmax (attn_weights = softmax(masked_scores))
        # dL/dx = y * (dL/dy - sum(dL/dy * y)) where y = softmax(x)
        grad_masked_scores = grad_attn_weights * attn_weights - np.sum(grad_attn_weights * attn_weights, axis=-1, keepdims=True) * attn_weights

        # 7. Backward through scores + causal_mask (causal_mask is constant, so its gradient is 0)
        grad_scores = grad_masked_scores

        # 8. Backward through scaled dot-product: scores = (Q @ K.T) / sqrt(d_k)
        # Let S = Q @ K.T / sqrt(d_k)
        # dL/dQ = (dL/dS @ K) / sqrt(d_k)
        # dL/dK = (dL/dS.T @ Q) / sqrt(d_k)
        grad_Q = np.matmul(grad_scores, K) / math.sqrt(self.d_k)
        grad_K = np.matmul(grad_scores.transpose(0, 1, 3, 2), Q) / math.sqrt(self.d_k)

        # 9. Undo split_heads for Q, K, V to get gradients for original Q_orig, K_orig, V_orig
        def un_split_heads(z_grad, original_shape):
            B_s, NH_s, T_s, DK_s = z_grad.shape
            z_grad = z_grad.transpose(0, 2, 1, 3) # (B, T, n_heads, d_k)
            return z_grad.reshape(original_shape) # (B, T, head_size)

        grad_Q_orig = un_split_heads(grad_Q, Q_orig.shape)
        grad_K_orig = un_split_heads(grad_K, K_orig.shape)
        grad_V_orig = un_split_heads(grad_V, V_orig.shape)

        # 10. Backward through q_proj, k_proj, v_proj
        grad_x_q, q_proj_grads = self.q_proj.backward(grad_Q_orig)
        grad_x_k, k_proj_grads = self.k_proj.backward(grad_K_orig)
        grad_x_v, v_proj_grads = self.v_proj.backward(grad_V_orig)

        # Sum gradients for the input 'x' from all three paths (Q, K, V)
        grad_x = grad_x_q + grad_x_k + grad_x_v

        # Assemble gradients in the correct order: q_proj, k_proj, v_proj, c_proj
        current_mha_param_grads.extend(q_proj_grads)
        current_mha_param_grads.extend(k_proj_grads)
        current_mha_param_grads.extend(v_proj_grads)
        current_mha_param_grads.extend(c_proj_grads)

        return grad_x, current_mha_param_grads

In [ ]:
########################
# Multi-Layer Perceptron (MLP)
########################

class MLP(Module):
    """
    Multi-Layer Perceptron block, typically used in Transformer after attention.
    Consists of two linear layers with GELU activation and dropout.
    """
    def __init__(self, n_emb, dropout_p):
        super().__init__()
        # First linear layer (expands dimension)
        self.c_fc = Linear(n_emb, 4 * n_emb)
        # Second linear layer (projects back to original dimension)
        self.c_proj = Linear(4 * n_emb, n_emb)
        self.dropout = Dropout(dropout_p)

    def parameters(self):
        """Returns all parameters of the MLP module."""
        return self.c_fc.parameters() + self.c_proj.parameters()

    def set(self, mode=True):
        """Sets the MLP module and its sub-modules to training/eval mode."""
        super().set(mode)
        self.c_fc.set(mode)
        self.c_proj.set(mode)
        self.dropout.set(mode)

    def forward(self, x):
        """
        Forward pass for MLP.
        x: input tensor, shape (B, T, n_emb)
        Returns: output tensor, shape (B, T, n_emb)
        """
        self._cache_x = x # Store input to MLP for backward pass

        fc_out = self.c_fc.forward(x)
        gelu_out = gelu(fc_out)
        proj_out = self.c_proj.forward(gelu_out)
        dropped_out = self.dropout.forward(proj_out)

        # Store intermediate results for backward pass
        self._cache = (fc_out, gelu_out, proj_out)
        return dropped_out

    def backward(self, grad_output):
        """
        Backward pass for MLP.
        grad_output: gradient from subsequent layer.
        Returns: (grad_input, list_of_param_grads)
        """
        x = self._cache_x
        fc_out, gelu_out, proj_out = self._cache

        # Gradients will be collected in the order of self.parameters(): c_fc, c_proj
        current_mlp_param_grads = []

        # 1. Backward through dropout
        grad_proj_out, _ = self.dropout.backward(grad_output)

        # 2. Backward through c_proj
        grad_gelu_out, c_proj_grads = self.c_proj.backward(grad_proj_out)


        # 3. Backward through GELU activation
        # dL/dx = dL/dy * gelu_prime(x)
        grad_fc_out = grad_gelu_out * gelu_prime(fc_out)

        # 4. Backward through c_fc
        grad_x, c_fc_grads = self.c_fc.backward(grad_fc_out)

        # Assemble gradients in the correct order: c_fc, c_proj
        current_mlp_param_grads.extend(c_fc_grads)
        current_mlp_param_grads.extend(c_proj_grads)

        return grad_x, current_mlp_param_grads

In [ ]:
########################
# Transformer Block
########################

class Block(Module):
    """
    A single Transformer Block.
    Consists of LayerNorm, MultiHeadAttention, another LayerNorm, and an MLP.
    Includes residual connections.
    """
    def __init__(self, n_emb, head_size, n_heads, n_ctx, dropout_p):
        super().__init__()
        self.ln_1 = LayerNorm(n_emb)
        self.mha = MultiHeadAttention(n_emb, head_size, n_heads, n_ctx, dropout_p)
        self.ln_2 = LayerNorm(n_emb)
        self.mlp = MLP(n_emb, dropout_p)

    def parameters(self):
        """Returns all parameters of the Transformer Block."""
        return (self.ln_1.parameters() +
                self.mha.parameters() + # Order changed for consistency
                self.ln_2.parameters() +
                self.mlp.parameters())

    def set(self, mode=True):
        """Sets the block and its sub-modules to training/eval mode."""
        super().set(mode)
        self.ln_1.set(mode)
        self.ln_2.set(mode)
        self.mha.set(mode)
        self.mlp.set(mode)

    def forward(self, x, use_cache):
        """
        Forward pass for a Transformer Block.
        x: input tensor, shape (B, T, n_emb)
        Returns: output tensor, shape (B, T, n_emb)
        """
        self._cache_x = x # Store input for the first residual connection

        # First residual connection: x + MHA(LayerNorm(x))
        ln1_out = self.ln_1.forward(x)
        mha_out = self.mha.forward(ln1_out, use_cache)
        x_res1 = x + mha_out # Residual connection 1

        # Second residual connection: x_res1 + MLP(LayerNorm(x_res1))
        ln2_out = self.ln_2.forward(x_res1)
        mlp_out = self.mlp.forward(ln2_out)
        out = x_res1 + mlp_out # Residual connection 2

        # Store intermediate values for backward pass
        self._cache = (ln1_out, mha_out, ln2_out, mlp_out, x_res1)
        return out

    def backward(self, grad_output):
        """
        Backward pass for a Transformer Block.
        grad_output: gradient from subsequent layer.
        Returns: (grad_input, list_of_param_grads)
        """
        x = self._cache_x
        ln1_out, mha_out, ln2_out, mlp_out, x_res1 = self._cache

        # Gradients will be collected in the order of self.parameters(): ln_1, mha, ln_2, mlp
        current_block_param_grads = []

        # 1. Backward through second residual connection and MLP
        grad_x_res1_from_res2 = grad_output
        grad_mlp_out = grad_output

        grad_ln2_out, mlp_grads = self.mlp.backward(grad_mlp_out)

        grad_x_res1_from_ln2, ln2_grads = self.ln_2.backward(grad_ln2_out)

        # Sum gradients for x_res1 from both paths
        grad_x_res1 = grad_x_res1_from_res2 + grad_x_res1_from_ln2

        # 2. Backward through first residual connection and MHA
        grad_x_from_res1 = grad_x_res1
        grad_mha_out = grad_x_res1

        grad_ln1_out, mha_grads = self.mha.backward(grad_mha_out)

        grad_x_from_ln1, ln1_grads = self.ln_1.backward(grad_ln1_out)

        # Sum gradients for the initial input 'x' from both paths
        grad_x = grad_x_from_res1 + grad_x_from_ln1

        # Assemble gradients in the correct order: ln_1, mha, ln_2, mlp
        current_block_param_grads.extend(ln1_grads)
        current_block_param_grads.extend(mha_grads)
        current_block_param_grads.extend(ln2_grads)
        current_block_param_grads.extend(mlp_grads)

        return grad_x, current_block_param_grads

In [ ]:
########################
# Optimizer
########################

class AdamW:
    """
    AdamW optimizer implementation.
    Includes adaptive learning rates and weight decay.
    """
    def __init__(self, parameters, learning_rate=1e-3, beta1=0.9, beta2=0.999, eps=1e-8, weight_decay=0.01):
        self.params = parameters # List of all trainable parameters (NumPy arrays)
        self.lr = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.weight_decay = weight_decay
        self.t = 0 # Timestep counter

        # Initialize first and second moment estimates for each parameter
        self.m = {id(p): np.zeros_like(p) for p in self.params}
        self.v = {id(p): np.zeros_like(p) for p in self.params}

    def step(self, grads):
        """
        Performs a single optimization step (parameter update).
        grads: a list of gradients, corresponding to self.params.
        """
        self.t += 1 # Increment timestep
        for p, g in zip(self.params, grads):
            pid = id(p) # Use object ID for unique parameter identification

            # Skip weight decay for 1D params (biases, LayerNorm gamma/beta)
            if self.weight_decay and p.ndim > 1:
                # Apply weight decay (L2 regularization)
                # This is applied directly to the gradient before the Adam update
                g = g + self.weight_decay * p

            # Update biased first moment estimate
            self.m[pid] = self.beta1 * self.m[pid] + (1 - self.beta1) * g
            # Update biased second raw moment estimate
            self.v[pid] = self.beta2 * self.v[pid] + (1 - self.beta2) * (g * g)

            # Compute bias-corrected first moment estimate
            m_hat = self.m[pid] / (1 - self.beta1 ** self.t)
            # Compute bias-corrected second raw moment estimate
            v_hat = self.v[pid] / (1 - self.beta2 ** self.t)

            # Update parameters
            p -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)

In [ ]:
########################
# Loss Function
########################

def cross_entropy_loss(logits, targets):
    """
    Computes cross-entropy loss and its gradient with respect to logits.
    logits: (B, T, vocab_size) - raw predictions from the model
    targets: (B, T) - true token IDs
    Returns: (loss_value, grad_logits)
    """
    B, T, C = logits.shape
    logits_flat = logits.reshape(B * T, C)
    targets_flat = targets.reshape(B * T)

    # For numerical stability: subtract max logit from all logits before exponentiation
    logits_max = np.max(logits_flat, axis=1, keepdims=True)
    exp_logits = np.exp(logits_flat - logits_max)

    sum_exp_logits = np.sum(exp_logits, axis=1, keepdims=True)
    probs = exp_logits / sum_exp_logits # Softmax probabilities

    # Compute loss: - sum(target_one_hot * log(probs))
    log_probs = np.log(probs + 1e-9) # Add epsilon for numerical stability to avoid log(0)
    loss = -np.mean(log_probs[np.arange(B * T), targets_flat])

    # Compute gradient of cross-entropy loss with respect to logits
    # The derivative of Cross-Entropy + Softmax is (probs - one_hot_targets)
    one_hot_targets = np.zeros_like(probs)
    one_hot_targets[np.arange(B * T), targets_flat] = 1

    grad_logits = probs - one_hot_targets # Shape: (B*T, C)
    grad_logits = grad_logits.reshape(B, T, C) # Reshape back to (B, T, C)

    return loss, grad_logits

In [ ]:
########################
# Calculate Value and Grand
########################

def value_and_grad(model, x, y):
    """
    Performs a forward pass to compute loss and then a backward pass
    to compute gradients for all model parameters.
    """
    # Forward pass
    logits = model.forward(x, use_cache=False)
    # Compute loss and get initial gradient for logits from the loss function
    loss, grad_logits = cross_entropy_loss(logits, y)

    # Backward pass: The model's backward method takes the gradient from the loss
    # and propagates it back through all layers, returning gradients for parameters.
    _, grads = model.backward(grad_logits) # grad_input for model is None

    return loss, grads

def value_and_nograd(model, x, y):
    """
    Performs a forward pass to compute loss
    """
    # Forward pass
    logits = model.forward(x, use_cache=False)
    loss, _ = cross_entropy_loss(logits, y) 

    return loss, _

In [ ]:
########################
# GPT Model Definition
########################

class GPT(Module):
    """
    A minimal GPT (Generative Pre-trained Transformer) model.
    """
    def __init__(self, n_ctx, n_emb, n_layers, head_size, n_heads, dropout_p):
        super().__init__()
        self.n_ctx = n_ctx
        self.vocab_size = 1 # Placeholder for vocabulary size, should updated
        self.wte = Embedding(self.vocab_size, n_emb)    # Token embeddings
        self.wpe = Embedding(n_ctx, n_emb)        # Positional embeddings
        self.blocks = [Block(n_emb, head_size, n_heads, n_ctx, dropout_p)
                       for _ in range(n_layers)]    # Stack of Transformer blocks
        self.ln_f = LayerNorm(n_emb)                # Final Layer Normalization
        self.lm_head = Linear(n_emb, self.vocab_size)    # Language modeling head (output logits)

    def parameters(self):
        """Returns all parameters of the GPT model."""
        params = []
        params += self.wte.parameters()
        params += self.wpe.parameters()
        for block in self.blocks:
            params += block.parameters()
        params += self.ln_f.parameters()
        params += self.lm_head.parameters()
        return params

    def set(self, mode=True):
        """Sets the GPT model and all its sub-modules to training/eval mode."""
        super().set(mode) # Call base Module train to set self.set
        self.wte.set(mode)
        self.wpe.set(mode)
        for block in self.blocks:
            block.set(mode)
        self.ln_f.set(mode)
        self.lm_head.set(mode)

    def clear_cache(self):
        """Clear all KV caches in the model."""
        for block in self.blocks:
            block.mha.clear_cache()
        if hasattr(self, '_position_offset'):
            del self._position_offset

    def forward(self, x, use_cache):
        """
        Forward pass for the GPT model.
        x: input token IDs, shape (B, T)
        Returns: logits, shape (B, T, vocab_size)
        """
        B, T = x.shape
        tok_emb = self.wte.forward(x)   # (B, T, n_emb)
        
        # For KV cache, we need to offset position indices
        if use_cache and hasattr(self, '_position_offset'):
            # Calculate starting position with modulo wrap
            pos_start = self._position_offset % self.n_ctx
            pos_indices = np.arange(T) + pos_start
            pos_indices = pos_indices % self.n_ctx  # Wrap around if needed
        else:
            pos_indices = np.arange(T) % self.n_ctx # Wrap around if needed

        pos_emb = self.wpe.forward(pos_indices) # (T, n_emb)
        
        # Combine token and position embeddings (positional embeddings are broadcasted)
        x_combined_emb = tok_emb + pos_emb

        # Pass through Transformer blocks
        current_x = x_combined_emb

        # We need to store the output of each block to correctly backpropagate through the sequential blocks.
        # However, the Block's backward method only needs its *own* input gradient, not the full history.
        # The chain rule handles this sequentially.
        for block in self.blocks:
            current_x = block.forward(current_x, use_cache)

        ln_f_out = self.ln_f.forward(current_x)
        logits = self.lm_head.forward(ln_f_out)  # (B, T, vocab_size)

        # Update position offset for next iteration
        if use_cache:
            if not hasattr(self, '_position_offset'):
                self._position_offset = T
            else:
                self._position_offset += T

        # Store intermediate values for backward pass
        self._cache = (x_combined_emb, current_x, ln_f_out)
        return logits

    def backward(self, grad_output):
        """
        Backward pass for the GPT model.
        grad_output: gradient from the loss function, shape (B, T, vocab_size)
        Returns: (None, list_of_param_grads) - no grad_input for the whole model.
        """
        (x_combined_emb, current_x_before_lnf, ln_f_out) = self._cache

        # Initialize an empty list to store gradients in the correct order
        # The order must match self.parameters()
        ordered_param_grads = []

        # 1. Backward through lm_head
        grad_ln_f_out, lm_head_grads = self.lm_head.backward(grad_output)

        # 2. Backward through ln_f (final LayerNorm)
        grad_current_x_before_lnf, ln_f_grads = self.ln_f.backward(grad_ln_f_out)

        # 3. Backward through blocks in reverse order
        # Need to store block gradients temporarily in correct order (forward pass order)
        # to match how self.blocks are added in parameters()
        block_grads_temp = [None] * len(self.blocks) # Temporary storage for block gradients
        grad_for_prev_block = grad_current_x_before_lnf
        for i in reversed(range(len(self.blocks))):
            block = self.blocks[i]
            grad_for_prev_block, current_block_grads = block.backward(grad_for_prev_block)
            block_grads_temp[i] = current_block_grads # Store in forward order index

        # 4. Backward through token + position embeddings addition
        # grad_for_prev_block is now the gradient for x_combined_emb (tok_emb + pos_emb)
        grad_tok_emb = grad_for_prev_block # Gradient for token embeddings
        # For position embeddings, sum gradients over the batch dimension
        grad_pos_emb = np.sum(grad_for_prev_block, axis=0)

        # 5. Backward through wte (token embeddings) and wpe (position embeddings)
        # Embedding.backward returns (None, [grad_weight])
        _, wte_grads = self.wte.backward(grad_tok_emb)
        _, wpe_grads = self.wpe.backward(grad_pos_emb)

        # Now, assemble the ordered_param_grads list in the same order as self.parameters()
        ordered_param_grads.extend(wte_grads)
        ordered_param_grads.extend(wpe_grads)
        for grads_list_for_block in block_grads_temp: # These are already in forward order
            ordered_param_grads.extend(grads_list_for_block)
        ordered_param_grads.extend(ln_f_grads)
        ordered_param_grads.extend(lm_head_grads)

        return None, ordered_param_grads # No grad_input for the entire model

    def generate(self, stoi, prompt, use_cache, max_new_tokens):
        """
        Generates new tokens based on the model's learned probabilities.
        prompt_ids: input sequence of token IDs to start generation.
        max_new_tokens: maximum number of tokens to generate.
        Returns: generated sequence of token IDs.
        """
        # Set model to evaluation mode for generation (disables dropout)
        self.eval()

        # Clear any existing cache
        self.clear_cache()
        
        # Start with a starting token (here we use index 0, assuming it's a valid token)
        # This could be a special <SOS> token in a more robust implementation.
        if prompt is None:
            ctx = np.zeros((1, 1), dtype=np.int32) # Initial context: a single token
        else:
            encode_ = lambda s: np.array([stoi[c] for c in s]).reshape(1, -1)
            prompt_ids = encode_(prompt)  # Your tokenizer should return shape (1, prompt_length)
            ctx = prompt_ids # Initial context: sequence of tokens

        # Process initial prompt (if any) without cache to establish context
        if ctx.shape[1] > 0:
            input_seq = ctx[:, -self.n_ctx:]
            _ = self.forward(input_seq, use_cache)

        for _ in range(max_new_tokens):
            if use_cache and ctx.shape[1] > 1:
                # For cached generation, only process the last token
                input_seq = ctx[:, -1:]
            else:
                # Use the last self.n_ctx tokens as input (or all if shorter)
                input_seq = ctx[:, -self.n_ctx:]

            logits = self.forward(input_seq, use_cache)

            # Get logits for the last token in the sequence (the one to predict)
            logits = logits[:, -1, :] # Shape: (1, vocab_size)

            # Convert logits to probabilities
            probs = softmax(logits, axis=-1).flatten() # Flatten to (vocab_size,)

            # Sample the next token based on probabilities
            next_tok = np.random.choice(np.arange(probs.shape[0]), p=probs)

            # Append the new token to the context
            ctx = np.concatenate([ctx, np.array([[next_tok]], dtype=np.int32)], axis=1)

        return ctx

    def expand_embeddings(self, new_vocab_size):
        """ Expands the token embeddings and output layer to accommodate a new vocabulary size. """

        # old_vocab_size is set to 1 for initial training
        # old_vocab_size could be smaller than new_vocab_size for fine tuning
        old_vocab_size = self.vocab_size
        
        if new_vocab_size > old_vocab_size:
            print(f"Expanding model embeddings from {old_vocab_size} to {new_vocab_size}.")
            # Expand token embeddings
            self.vocab_size = new_vocab_size
            new_wte_weight = np.random.randn(new_vocab_size, self.wte.weight.shape[1]) * 0.02
            new_wte_weight[:old_vocab_size] = self.wte.weight
            self.wte.weight = new_wte_weight
            
            # Expand output layer
            new_lm_head_weight = np.random.randn(self.lm_head.weight.shape[0], new_vocab_size) * 0.02
            new_lm_head_weight[:, :old_vocab_size] = self.lm_head.weight
            self.lm_head.weight = new_lm_head_weight
            
            if self.lm_head.bias is not None:
                new_bias = np.zeros(new_vocab_size)
                new_bias[:old_vocab_size] = self.lm_head.bias
                self.lm_head.bias = new_bias

            self.wte._parameters = [self.wte.weight]
            self.lm_head._parameters = [self.lm_head.weight]
            if self.lm_head.bias is not None:
                self.lm_head._parameters.append(self.lm_head.bias)
    
    def load(self, model_path):
        """ Load model weights from a JSON file. """

        print("--- Start loading ---")
        json_weights_dict = from_file(model_path, "json")

        # Convert lists back to numpy arrays
        weights_dict = {}
        for key, value in json_weights_dict.items():
            if value is not None and isinstance(value, list):
                weights_dict[key] = np.array(value, dtype=np.float32)
            else:
                weights_dict[key] = value

        # Update vocab_size from loaded weights BEFORE restoring weights
        self.vocab_size = weights_dict['wte_weight'].shape[0]

        # Restore weights to the model
        self.wte.weight = weights_dict['wte_weight']
        self.wpe.weight = weights_dict['wpe_weight']
        self.ln_f.gamma = weights_dict['ln_f_gamma']
        self.ln_f.beta = weights_dict['ln_f_beta']
        self.lm_head.weight = weights_dict['lm_head_weight']
        if weights_dict['lm_head_bias'] is not None:
            self.lm_head.bias = weights_dict['lm_head_bias']

        # UPDATE the main model's top-level _parameters
        self.wte._parameters = [self.wte.weight]
        self.wpe._parameters = [self.wpe.weight]
        self.ln_f._parameters = [self.ln_f.gamma, self.ln_f.beta]
        self.lm_head._parameters = [self.lm_head.weight]
        if self.lm_head.bias is not None:
            self.lm_head._parameters.append(self.lm_head.bias)
        
        # Restore block weights
        for i, block in enumerate(self.blocks):
            # Layer norms
            block.ln_1.gamma = weights_dict[f'block_{i}_ln1_gamma']
            block.ln_1.beta = weights_dict[f'block_{i}_ln1_beta']
            block.ln_2.gamma = weights_dict[f'block_{i}_ln2_gamma']
            block.ln_2.beta = weights_dict[f'block_{i}_ln2_beta']

            # UPDATE LayerNorm _parameters
            block.ln_1._parameters = [block.ln_1.gamma, block.ln_1.beta]
            block.ln_2._parameters = [block.ln_2.gamma, block.ln_2.beta]
            
            # Multi-head attention
            block.mha.q_proj.weight = weights_dict[f'block_{i}_mha_q_weight']
            block.mha.k_proj.weight = weights_dict[f'block_{i}_mha_k_weight']
            block.mha.v_proj.weight = weights_dict[f'block_{i}_mha_v_weight']
            block.mha.c_proj.weight = weights_dict[f'block_{i}_mha_c_weight']
            if weights_dict[f'block_{i}_mha_c_bias'] is not None:
                block.mha.c_proj.bias = weights_dict[f'block_{i}_mha_c_bias']

            # UPDATE MHA projection _parameters
            block.mha.q_proj._parameters = [block.mha.q_proj.weight]
            block.mha.k_proj._parameters = [block.mha.k_proj.weight]
            block.mha.v_proj._parameters = [block.mha.v_proj.weight]
            block.mha.c_proj._parameters = [block.mha.c_proj.weight]
            if block.mha.c_proj.bias is not None:
                block.mha.c_proj._parameters.append(block.mha.c_proj.bias)
            
            # MLP
            block.mlp.c_fc.weight = weights_dict[f'block_{i}_mlp_fc_weight']
            if weights_dict[f'block_{i}_mlp_fc_bias'] is not None:
                block.mlp.c_fc.bias = weights_dict[f'block_{i}_mlp_fc_bias']
            block.mlp.c_proj.weight = weights_dict[f'block_{i}_mlp_proj_weight']
            if weights_dict[f'block_{i}_mlp_proj_bias'] is not None:
                block.mlp.c_proj.bias = weights_dict[f'block_{i}_mlp_proj_bias']

            # UPDATE MLP _parameters
            block.mlp.c_fc._parameters = [block.mlp.c_fc.weight]
            if block.mlp.c_fc.bias is not None:
                block.mlp.c_fc._parameters.append(block.mlp.c_fc.bias)
            block.mlp.c_proj._parameters = [block.mlp.c_proj.weight]
            if block.mlp.c_proj.bias is not None:
                block.mlp.c_proj._parameters.append(block.mlp.c_proj.bias)
                
        print("--- Stop loading ---")

    def save(self, model_path):     
        """ Save model weights to a JSON file. """

        print("--- Start saving ---")

        # Extract all weight arrays from the model
        weights_dict = {
            # Token and position embeddings
            'wte_weight': self.wte.weight,
            'wpe_weight': self.wpe.weight,
            
            # Final layer norm
            'ln_f_gamma': self.ln_f.gamma,
            'ln_f_beta': self.ln_f.beta,
            
            # Language model head
            'lm_head_weight': self.lm_head.weight,
            'lm_head_bias': self.lm_head.bias if self.lm_head.bias is not None else None,
        }
        
        # Add block weights
        for i, block in enumerate(self.blocks):
            # Layer norms
            weights_dict[f'block_{i}_ln1_gamma'] = block.ln_1.gamma
            weights_dict[f'block_{i}_ln1_beta'] = block.ln_1.beta
            weights_dict[f'block_{i}_ln2_gamma'] = block.ln_2.gamma
            weights_dict[f'block_{i}_ln2_beta'] = block.ln_2.beta
            
            # Multi-head attention
            weights_dict[f'block_{i}_mha_q_weight'] = block.mha.q_proj.weight
            weights_dict[f'block_{i}_mha_k_weight'] = block.mha.k_proj.weight
            weights_dict[f'block_{i}_mha_v_weight'] = block.mha.v_proj.weight
            weights_dict[f'block_{i}_mha_c_weight'] = block.mha.c_proj.weight
            weights_dict[f'block_{i}_mha_c_bias'] = block.mha.c_proj.bias if block.mha.c_proj.bias is not None else None
            
            # MLP
            weights_dict[f'block_{i}_mlp_fc_weight'] = block.mlp.c_fc.weight
            weights_dict[f'block_{i}_mlp_fc_bias'] = block.mlp.c_fc.bias if block.mlp.c_fc.bias is not None else None
            weights_dict[f'block_{i}_mlp_proj_weight'] = block.mlp.c_proj.weight
            weights_dict[f'block_{i}_mlp_proj_bias'] = block.mlp.c_proj.bias if block.mlp.c_proj.bias is not None else None
        
        # Convert numpy arrays to lists for JSON serialization
        json_weights_dict = {}
        for key, value in weights_dict.items():
            if value is not None and hasattr(value, 'tolist'):
                json_weights_dict[key] = value.tolist()
            else:
                json_weights_dict[key] = value

        # Save weights only
        to_file(model_path, "json", json_weights_dict)

        print("--- Stop saving ---")

    def train(self, input_path, tokenizer_path, report_path, n_epochs, batch_size, lr):
        """ Train the GPT model on the provided input data. """
        print("--- Start training ---")

        # Create tokenizer
        tokenizer = CharTokenizer()
        
        # Tokenize the input data
        train_data, val_data = tokenizer.tokenize(input_path)
        
        X_train, y_train = tokenizer.prepare_data(train_data, self.n_ctx)
        X_val, y_val = tokenizer.prepare_data(val_data, self.n_ctx)

        # If vocabulary expanded, resize model embeddings
        vocab_size = tokenizer.vocab_size  # Get the size of the vocabulary from the tokenizer
        if vocab_size > self.vocab_size: # If the vocabulary size has changed, expand embeddings
            self.expand_embeddings(vocab_size)

        params = self.parameters() # Get all trainable parameters from the model

        optimizer = AdamW(params, learning_rate=lr)

        batch_logs = []
        epoch_logs = []
        epoch_total_time = 0
        total_time = 0

        for epoch in range(n_epochs):
            # Record start time
            epoch_start_time = time.time()

            # Training phase
            self.set(True) # Set model to training mode (enables dropout)

            running_train_loss = 0
            train_batch_cnt = 0
            train_batch_all = 0

            train_batches = list(tokenizer.get_batches(X_train, y_train, batch_size, shuffle=True))
            train_batch_all = len(train_batches)
            if not train_batches:
                print(f"Epoch {epoch:2}/{n_epochs:2} | No training data batches available. Skipping training for this epoch.")
                avg_train_loss = float('nan')
            else:
                for X_batch, y_batch in train_batches:
                    train_batch_cnt += 1
                    train_batch_start_time = time.time()  # Record start time
                    # Compute loss and gradients
                    loss, grads = value_and_grad(self, X_batch, y_batch)
                    # Update model parameters using the optimizer
                    optimizer.step(grads)
                    running_train_loss += loss
                    train_batch_stop_time = time.time()  # Record stop time
                    train_batch_elapsed_time = train_batch_stop_time - train_batch_start_time
                    # Print loss for the current batch
                    batch_log = f"Epoch {epoch:2}/{n_epochs:2} | Batch {train_batch_cnt:2}/{train_batch_all:2} | train_loss = {loss:.4f} | execution_time = {train_batch_elapsed_time:.4f}"
                    batch_logs.append(batch_log)
                    print(batch_log)
                avg_train_loss = running_train_loss / train_batch_cnt

            # Validation phase
            self.set(False) # Set model to evaluation mode (disables dropout)

            running_val_loss = 0
            val_batch_cnt = 0
            val_batch_all = 0

            val_batches = list(tokenizer.get_batches(X_val, y_val, batch_size, shuffle=False))
            val_batch_all = len(val_batches)
            if not val_batches:
                print(f"Epoch {(epoch+1):2}/{n_epochs:2} | No validation data batches available. Skipping validation for this epoch.")
                avg_val_loss = float('nan') # Indicate no validation was performed
            else:
                for X_batch, y_batch in val_batches:
                    val_batch_cnt += 1
                    val_batch_start_time = time.time()  # Record start time
                    # In validation, only forward pass and loss computation are needed
                    loss, _ = value_and_nograd(self, X_batch, y_batch)
                    running_val_loss += loss
                    val_batch_stop_time = time.time()  # Record stop time
                    val_batch_elapsed_time = val_batch_stop_time - val_batch_start_time  
                    # Print loss for the current batch
                    batch_log = f"Epoch {epoch:2}/{n_epochs:2} | Batch {val_batch_cnt:2}/{val_batch_all:2} | val_loss = {loss:.4f} | execution_time = {val_batch_elapsed_time:.4f}"
                    batch_logs.append(batch_log)
                    print(batch_log)
                avg_val_loss = running_val_loss / val_batch_cnt

            # Record stop time
            epoch_stop_time = time.time()

            # Calculate elapsed time for the epoch
            epoch_elapsed_time = epoch_stop_time - epoch_start_time  
            epoch_total_time += epoch_elapsed_time

            # Print average losses for the epoch
            epoch_log = f"Epoch {(epoch+1):2}/{n_epochs:2} | train_loss = {avg_train_loss:.4f} | val_loss = {avg_val_loss:.4f} | execution_time = {epoch_elapsed_time:.4f}"
            epoch_logs.append(epoch_log)
            print(epoch_log)

        epoch_total_time_avg = epoch_total_time / n_epochs
        total_time += epoch_total_time
        print(f"Average epoch time: {epoch_total_time_avg:.4f}")        

        # Create the report object
        report = {
            "num_epochs": n_epochs,
            "train_batches_per_epoch": train_batch_all,
            "val_batches_per_epoch": val_batch_all,
            "average_train_loss_per_epoch": avg_train_loss,
            "average_val_loss_per_epoch": avg_val_loss,
            "average_time_per_epoch": epoch_total_time_avg,
            "batch_logs": batch_logs,
            "epoch_logs": epoch_logs,
            "total_time": total_time
        }

        # Save the report to a JSON file
        to_file(report_path, "json", report)

        #Create the tokenizer object
        tokenizer_json = tokenizer.to_dict()

        # Save the tokenizer to a JSON file
        to_file(tokenizer_path, "json", tokenizer_json)
        
        print("--- Stop training ---")

    def inference(self, prompt, kv_cache, tokenizer_path, completion_path):
        """ Perform inference with the trained model using a given prompt. """
        print("--- Start Inference ---")

        # Create tokenizer
        tokenizer = CharTokenizer()
        tokenizer_dict = from_file(tokenizer_path, "json")
        tokenizer.from_dict(tokenizer_dict)
        self.vocab_size = tokenizer.vocab_size  # Update model vocabulary size from tokenizer
        
        # If vocabulary expanded, resize model embeddings / Is not needed as load is loaded expanded matixes
        # vocab_size = tokenizer.vocab_size  # Get the size of the vocabulary from the tokenizer
        # if vocab_size > self.vocab_size: # If the vocabulary size has changed, expand embeddings
        #     self.expand_embeddings(vocab_size)

        # Load the tokenizer's string-to-index mapping
        stoi = tokenizer.stoi
        
        # Initialize the total inference time
        inference_total_time = 0

        # Generate 500 new tokens
        inference_max_tokens = 500

        # Record start time
        inference_start_time = time.time()

        # Generate text based on given token ids
        generation_ids = self.generate(stoi, prompt, kv_cache, inference_max_tokens)
        
        # Decode the generated token IDs back to text
        generation = tokenizer.decode(generation_ids[0].tolist())

        # Record stop time
        inference_stop_time = time.time() 

        # Calculate elapsed time for inference
        inference_elapsed_time = inference_stop_time - inference_start_time  
        inference_total_time += inference_elapsed_time

        # Create a completion object with the prompt, generated text, and inference time
        completion = { "prompt": prompt, "generation": generation, "inference_max_tokens": inference_max_tokens, "inference_total_time": inference_total_time }

        print(completion)

        # Save the completion text to a JSON file        
        to_file(completion_path, "json", completion)

        print("--- Stop Inference ---")

In [ ]:
########################
# Runtime configuration
########################

runtime_path_inp = input("Enter the runtime path ('same', '<path>'): ").strip().lower()
runtime_plan_inp = input("Enter the runtime plan ('train', 'finetune', 'inference'): ").strip().lower()

In [ ]:
########################
# Runtime variables
########################

if runtime_path_inp == "same":
    runtime_path = "."
else:
    runtime_path = runtime_path_inp
    
runtime_plan = runtime_plan_inp

configs_path = f"{runtime_path}/configs"
tokenizers_path = f"{runtime_path}/tokenizers"
inputs_path = f"{runtime_path}/inputs"
outputs_path = f"{runtime_path}/outputs"
models_path = f"{runtime_path}/models"

In [ ]:
########################
# Runtime execution
########################

if runtime_plan == "train":
    # Generate a UUID (random UUID)
    runtime_uuid = uuid.uuid4()
    config_path = f"{configs_path}/current.json"
    config = from_file(config_path, "json")
elif runtime_plan == "finetune" or runtime_plan == "inference":
    # Provide the UUID or Last
    runtime_uuid_inp = input("Enter the model configuration ('<uuid>','last'): ").lower()

    # Load the last UUID and Prompt
    if runtime_uuid_inp == "last":
        config_path = f"{configs_path}/last.json"   
        last = from_file(config_path, "json")
        runtime_uuid_last = last['last_uuid']            
        prompt_last = last['last_prompt']
        runtime_uuid = runtime_uuid_last
    else:
        runtime_uuid = runtime_uuid_inp
    
    # Load the configuration from a file
    if is_valid_guid(runtime_uuid):
        config_path = f"{configs_path}/config_{runtime_uuid}.json"
        config = from_file(config_path, "json")
        print(f"Runtime GUID: {runtime_uuid}")
    else:
        print(f"Invalid UUID: {runtime_uuid}")
        exit(1)
else:
    print(f"Invalid model load plan: {runtime_plan}")
    exit(1)

In [ ]:
########################
# Runtime Flow
########################

# Instantiate the data
input_path = f"{inputs_path}/input.txt"

# Instantiate the additional settings
infr_cache = False #KV Cache

# Instantiate the model
model = GPT(config["n_ctx"], config["n_emb"], config["n_layers"], config["head_size"], config["n_heads"], config["dropout"])

if runtime_plan == "train":
    # Train the model from scratch
    n_epochs = config["num_epochs"]
    batch_size = config["batch_size"]
    lr = config["lr"]
    tokenizer_path = f"{tokenizers_path}/tokenizer_{runtime_uuid}.json"
    report_path = f"{outputs_path}/report_{runtime_uuid}.json"
    model.train(input_path, tokenizer_path, report_path, n_epochs, batch_size, lr)
    # Save the trained model weights
    model_path = f"{models_path}/model_{runtime_uuid}.weights"
    model.save(model_path)
    # Save the runtime settings text to a file
    config_path = f"{configs_path}/config_{runtime_uuid}.json"
    to_file(config_path, "json", config)
    # Inference using the trained model
    prompt = None
    tokenizer_path = f"{tokenizers_path}/tokenizer_{runtime_uuid}.json"
    completion_path = f"{outputs_path}/completion_{runtime_uuid}.json"
    model.inference(prompt, infr_cache, tokenizer_path, completion_path)
elif runtime_plan == "finetune":
    # Load the pre-trained model weights
    model_path = f"{models_path}/model_{runtime_uuid}.weights"
    model.load(model_path)
    # Fine Tune the model with the new data
    n_epochs = config["num_epochs"]
    batch_size = config["batch_size"]
    lr = config["lr"]
    tokenizer_path = f"{tokenizers_path}/tokenizer_{runtime_uuid}_finetuned.json"
    report_path = f"{outputs_path}/report_{runtime_uuid}.json"
    model.train(input_path, tokenizer_path, report_path, n_epochs, batch_size, lr)
    # Save the fine-tuned model weights
    model_path = f"{models_path}/model_{runtime_uuid}_finetuned.weights"
    model.save(model_path)
    # Inference using the fine-tuned model
    prompt = None
    tokenizer_path = f"{tokenizers_path}/tokenizer_{runtime_uuid}_finetuned.json"
    completion_path = f"{outputs_path}/completion_{runtime_uuid}_finetuned.json"
    model.inference(prompt, infr_cache, tokenizer_path, completion_path)
elif runtime_plan == "inference":
    prompt_inpt = input("Enter the prompt for inference ('<prompt>', 'last', 'none'): ").lower()
    if prompt_inpt == "last":
        prompt = prompt_last
    elif prompt_inpt == "none":
        prompt = None
    elif prompt_inpt == "":
        prompt = None
    else:
        prompt = prompt_inpt    
    # Load the pre-trained or fine-tuned model weights
    model_path = f"{models_path}/model_{runtime_uuid}.weights"
    model.load(model_path)
    # Inference with the model
    today = dt.today()
    today_ft = today.strftime('%Y%m%d%H%M%S')
    tokenizer_path = f"{tokenizers_path}/tokenizer_{runtime_uuid}.json"
    completion_path = f"{outputs_path}/completion_{runtime_uuid}_{today_ft}.json"
    model.inference(prompt, infr_cache, tokenizer_path, completion_path)

# Save the last settings text to a file
last = { "last_uuid": str(runtime_uuid), "last_prompt": str(prompt) }
config_path = f"{configs_path}/last.json"
to_file(config_path, "json", last)